In [2]:
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [3]:
target = 0                     # target for fitness function
nVar = 2                       # dimesion features
# ub = np.array([(0.1,0.8)])     # beta value range
# lb = np.array([(0.4,0.01)])   # gamma value range
ub = np.array([(2, 2)])     # max range for beta, gamma
lb = np.array([(0.01,0.01)])   # min range for beta, gamma

# hyper=paramters
noP = 5                        # no. of particles
maxIteration = 500             # Iterations

############ paramters for interia and acceleraion coefficients #########################
w = 0.5;
c1 = 2;
c2 = 2;
### W ###
# Linear time varying w_max = 0.9 , w_min = 0.4
w_min, w_max = 0.4, 0.9

# c1i = 2.5, c1f = 0.5, c2i= 0.5 and c2f = 2.5 are determined based on experiment experience
c1_i, c1_f = 2.5, 0.5
c2_i, c2_f = 0.5, 2.5

# velocity vector 
vMax = (ub-lb)*0.2;
vMin = -vMax;

#Initialization of particles 
SwarmParticlesX = np.zeros((noP,nVar),dtype=float)          # positons
SwarmParticlesV = np.zeros((noP,nVar),dtype=float)          # velocities

SwarmParticlesPBESTX = np.zeros((noP,nVar),dtype=float)     # personal best of X
SwarmParticlesPBESTObj = np.zeros((noP,1),dtype=float)      # personal best of fitness value

SwarmParticlesGBESTX = np.zeros((1,nVar),dtype=float)       # global best of swarm
SwarmGBESTObj = np.inf


cgCurve = np.zeros((maxIteration,1),dtype=float)

#for visulization
position_history = np.zeros((noP,maxIteration,nVar))

In [35]:
#helper functions
def coeffcient_c1_c2(iter_, c1_i=c1_i, c1_f=c1_f, c2_i=c2_i, c2_f=c2_f, iter_max = maxIteration):
    c1 = (c1_i - c1_f) *((iter_max - iter_)/ iter_max) + c1_f
    c2 = (c2_i - c2_f) *((iter_max - iter_)/ iter_max) + c2_f
    return c1, c2

#### Linear time varying  #####
def interia_w (iter_, w_min = w_min, w_max = w_max, iter_max = maxIteration):
    w = (w_min - w_max) * ((iter_max - iter_)/ iter_max) + w_max
    return w

#### non-linear time varying ####
def NLTV_interia_w(iter_, w_t):
    iter_max = maxIteration
    w = (w_t - 0.4)* ((iter_max - iter_)/ ( 0.4 + iter_max))
    return w

In [1]:
# Fitness function
def fitness_function(position):
    '''
    fitness function for curve fit. It uses measn squared error between true infected values and fitted values
    Parameters:
        ----------
        position: array

        Returns:
        ----------
        loss: float
    '''
    fitted = fit_odeint(t, position[0], position[1])
    loss = mean_squared_error(ydata, fitted)  
#     loss = mean_absolute_error(ydata, fitted)
    return int(loss)

In [39]:
def search_function():
    '''
    Calculates best value X(beta, gamma) using PSO as search algorithm for the curve fit

        Parameters:
        ----------
        (all parmeters are pre-defined)

        Returns:
        ----------
        SwarmParticlesGBESTX: array
        position_history: array
            the result (SwarmParticlesGBESTX) outputs best values of beta and gamma for curve fit
    
    '''
    
    # Initialization of PSO's parameter 
    gh = []
    SwarmGBESTObj= np.inf
    
    ### initalize particles
    for k in range(0,noP):
        SwarmParticlesX[k,:] = (ub - lb) * np.random.rand(1,nVar) + lb
        SwarmParticlesPBESTObj[k,0] = np.inf
   
    #### main PSO loop :
    for t in range(0,maxIteration):    
        
    #    ####### 1 #########
    #     for each particle:
    #         calculate fitness value
    #         if finess value better than pbest value in history then update it
        for k in range(0,noP):
            CurrentX = SwarmParticlesX[k,:]
            position_history[k,t,:] = CurrentX

            currento = fitness_function (CurrentX)

            if currento < SwarmParticlesPBESTObj[k,0]:
                SwarmParticlesPBESTX[k,:] = CurrentX
                SwarmParticlesPBESTObj[k,0] = currento

            if currento < SwarmGBESTObj:
                SwarmParticlesGBESTX = CurrentX
                SwarmGBESTObj = currento
    
                print('GBEST : {}'.format(CurrentX))
                gh.append(CurrentX)
                ### condition to exit loop when target reaches
                if SwarmGBESTObj == target:
                    print("Reached... target value")
         
                               
#     # new w, c1, c2 values
          
        if t == 0:
            w = 0.9
        w = NLTV_interia_w(t, w)
#         w = interia_w(t)
        c1, c2 = coeffcient_c1_c2(t)
        
        for k in range(0,noP):

            SwarmParticlesV[k,:] = (w * SwarmParticlesV[k,:]) + ((c1 * np.random.rand(1,nVar)) * (SwarmParticlesPBESTX[k,:]- SwarmParticlesX[k,:])) + ( (c2 * np.random.rand(1,nVar)) * (SwarmParticlesGBESTX - SwarmParticlesX[k,:])) 

            # Limit the Velocity Vector
            index1 = np.nonzero(SwarmParticlesV[k,:] > vMax)
            index2 = np.nonzero(SwarmParticlesV[k,:] < vMin)

            SwarmParticlesV[k,index1] = vMax[index1]
            SwarmParticlesV[k,index2] = vMin[index2]

            SwarmParticlesX[k,:] = SwarmParticlesX[k,:] + SwarmParticlesV[k,:]

            # Limit the Position Vector
            index1 = np.nonzero(SwarmParticlesX[k,:] > ub)
            index2 = np.nonzero(SwarmParticlesX[k,:] < lb)

            SwarmParticlesX[k,index1] = ub[index1]
            SwarmParticlesX[k,index2] = lb[index2]

        cgCurve[t,0] = SwarmGBESTObj
    print(SwarmParticlesGBESTX)
    return SwarmParticlesGBESTX, position_history, gh